In [1]:
import requests

session = requests.Session()
session.headers = {
    'application': 'LyricRec',
    'User-Agent': 'https://github.com/frankpacini/LyricRec'
}

access_token = 'Bearer ' + open("token2.txt", "r").read()
authorization_header = {'Authorization': access_token}

def getArtistId(artist_name):
    search_response = session.request('GET', 'https://api.genius.com/search', params={'q': artist_name, "per_page": 20}, headers=authorization_header)
    
    for i in range(50):
        search_result = search_response.json()['response']['hits'][i]['result']['primary_artist']
        search_result_name = search_result['name'].replace(u'\u200b', '')
        if search_result_name == artist_name:
            return search_result['id']
        else:
            print("    Incorrect artist name '{}', expected '{}'".format(search_result_name, artist_name))
    return -1

def getArtistSongs(artist_name, artist_id=None):
    if artist_id is None:
        artist_id = getArtistId(artist_name)
    uri = 'https://api.genius.com/artists/{}/songs'.format(artist_id)
    next_page = 1
    songs = []
    while next_page:
        params = {'sort': 'popularity', 'per_page': 50, 'page': next_page}
        response = session.request('GET', uri, params=params, headers=authorization_header)
        try:
            next_page = response.json()['response']['next_page']
        except KeyError:
            print(response.json())
        
        for song in response.json()['response']['songs']:
            if song['primary_artist']['name'] != artist_name or song['lyrics_state'] != 'complete':
                continue
            if 'pageviews' not in song['stats']:
                next_page = None
                break
            songs.append([song['id'], song['title'], artist_name, song['stats']['pageviews'], song['pyongs_count'], song['annotation_count']])
    return songs

In [7]:
with open("artists.txt", encoding="utf-8") as file:
    artists = {line.rstrip() for line in file.readlines()}
artists = list(artists)

In [8]:
ids = []
for artist in artists:
    print(artist)
    ids.append(getArtistId(artist))
artists += ['Prince', 'R.E.M.']
ids += [660, 8243]

Radiohead
Barbra Streisand
Bee Gees
Nirvana
Sly and the Family Stone
The Temptations
JAY-Z
Fleetwood Mac
Arcade Fire
King Crimson
Eagles
Swans
Simon & Garfunkel
Chris Brown
Taylor Swift
Marvin Gaye
    Incorrect artist name 'Charlie Puth', expected 'Marvin Gaye'
Otis Redding
Madvillain
James Brown
Eminem
Madonna
    Incorrect artist name 'Drake', expected 'Madonna'
Céline Dion
MF DOOM
    Incorrect artist name 'Madvillain', expected 'MF DOOM'
Animal Collective
Linkin Park
The Band
Katy Perry
Al Green
The Clash
Mariah Carey
Lou Reed
Patti Smith
    Incorrect artist name 'Patti Smith Group', expected 'Patti Smith'
Chuck Berry
Nick Drake
Buddy Holly
    Incorrect artist name 'Weezer', expected 'Buddy Holly'
Led Zeppelin
Stevie Wonder
Billy Joel
Björk
Van Morrison
Neil Young
Joy Division
The Doors
Metallica
Kate Bush
Tina Turner
    Incorrect artist name 'Ike & Tina Turner', expected 'Tina Turner'
Tool
The Who
Death Grips
The Jimi Hendrix Experience
OutKast
Bruno Mars
Elvis Presley
Oasis
T

In [14]:
import pandas as pd
pd.DataFrame({'Artist': artists, 'Id': ids}).to_csv("artists.csv")

In [16]:
artist_df = pd.read_csv("artists.csv")
songs = []
for _,row in artist_df.iterrows():
    print(row['Artist'])
    songs += getArtistSongs(row['Artist'], row['Id'])

Radiohead
Barbra Streisand
Bee Gees
Nirvana
Sly and the Family Stone
The Temptations
JAY-Z
Fleetwood Mac
Arcade Fire
King Crimson
Eagles
Swans
Simon & Garfunkel
Chris Brown
Taylor Swift
Marvin Gaye
Otis Redding
Madvillain
James Brown
Eminem
Madonna
Céline Dion
MF DOOM
Animal Collective
Linkin Park
The Band
Katy Perry
Al Green
The Clash
Mariah Carey
Lou Reed
Patti Smith
Chuck Berry
Nick Drake
Buddy Holly
Led Zeppelin
Stevie Wonder
Billy Joel
Björk
Van Morrison
Neil Young
Joy Division
The Doors
Metallica
Kate Bush
Tina Turner
Tool
The Who
Death Grips
The Jimi Hendrix Experience
OutKast
Bruno Mars
Elvis Presley
Oasis
The Rolling Stones
U2
Nick Cave & The Bad Seeds
Johnny Cash
Aerosmith
Ray Charles
Eric Clapton
A Tribe Called Quest
Maroon 5
Frank Zappa
Whitney Houston
Curtis Mayfield
Pink Floyd
The Police
Beastie Boys
The Beatles
James Taylor
The Byrds
Black Sabbath
Queen
Adele
my bloody valentine
Coldplay
Joni Mitchell
Lady Gaga
The Velvet Underground
Rihanna
The Cure
Lil Wayne
Aretha Fra

In [17]:
pd.DataFrame(songs, columns=['Id', 'Title', 'Artist', 'Page Views', 'Pyongs Count', 'Annotation Count']).to_csv('songs.csv')

In [136]:
import lyricsgenius as lg
import pandas as pd
TOKEN = open("token.txt", "r").read()
genius = lg.Genius(TOKEN, skip_non_songs=True, 
                   excluded_terms=["(Remix)", "(Live)", "- Live", "- Remix"], remove_section_headers=True)

In [145]:
df = pd.read_csv('songs_with_lyrics.csv').drop('Unnamed: 0', axis=1)
# start = 12571
# for i, row in list(df.iterrows())[start:]:
for i in list(df.loc[df['Lyrics'] == "''"].index):
    row = df.iloc[i]
    for j in range(5):
        try:
            lyrics = genius.lyrics(song_id=row['Id'])
            break
        except requests.Timeout:
            print("Timeout")
            # df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
            df.to_csv('songs_with_lyrics.csv')
            df = pd.read_csv('songs_with_lyrics.csv').drop('Unnamed: 0', axis=1)
            continue
    if j == 5:
        break
    if lyrics:
        lyrics_end = len(lyrics) - len("EmbedShare URLCopyEmbedCopy")
        while lyrics_end > 0:
            if not lyrics[lyrics_end-1].isdigit():
                break
            lyrics_end-=1
        df.at[i, 'Lyrics'] = repr(lyrics[:lyrics_end])
    print(i, row['Title'], row['Artist'])
df.to_csv('songs_with_lyrics.csv')
df.head()

Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Radiohead-treefingers-lyrics
91 Treefingers Radiohead
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Radiohead-hunting-bears-lyrics
106 Hunting Bears Radiohead
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Radiohead-untitled-lyrics
110 Untitled Radiohead
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Radiohead-meeting-in-the-aisle-lyrics
137 Meeting in the Aisle Radiohead
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Radiohead-mk-1-lyrics
144 MK 1 Radiohead
603 Purchases  Basquiat Painting ‘Mecca’ For $4.5M JAY-Z
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Arcade-fire-song-on-the-beach-l

Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Outkast-my-favorite-things-lyrics
4424 My Favorite Things OutKast
5264 Midnight Marauders [Artwork] A Tribe Called Quest
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Frank-zappa-peaches-en-regalia-lyrics
5407 Peaches en Regalia Frank Zappa
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Frank-zappa-apostrophe-lyrics
5411 Apostrophe’ Frank Zappa
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Pink-floyd-any-colour-you-like-lyrics
5522 Any Colour You Like Pink Floyd
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Pink-floyd-marooned-lyrics
5551 Marooned Pink Floyd
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: htt

9952 Florida Frank Ocean
9963 Street Fighter Frank Ocean
9978 Goldeneye Frank Ocean
9982 Soul Calibur Frank Ocean
9999 Channel ORANGE [Artwork] Frank Ocean
10012 Channel ORANGE [Booklet] Frank Ocean
10068 I feel u bro. Frank Ocean
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Kanye-west-all-of-the-lights-interlude-lyrics
10330 All of the Lights (Interlude) Kanye West
10418 The College Dropout [Booklet] Kanye West
10422 My Beautiful Dark Twisted Fantasy [Booklet] Kanye West
10430 Graduation [Booklet] Kanye West
10525 “Only One” Single Art Kanye West
10534 808s & Heartbreak [Booklet] Kanye West
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Red-hot-chili-peppers-pretty-little-ditty-lyrics
10831 Pretty Little Ditty Red Hot Chili Peppers
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Alice-in-chains-whale-and-wasp-

12931 Rock with You Michael Jackson
12932 The Way You Make Me Feel Michael Jackson
12933 You Are Not Alone Michael Jackson
12934 Remember the Time Michael Jackson
12935 You Rock My World Michael Jackson
12936 Scream Michael Jackson
12937 Off the Wall Michael Jackson
12938 The Girl Is Mine Michael Jackson
12939 Liberian Girl Michael Jackson
12940 Jam Michael Jackson
12941 I Just Can’t Stop Loving You Michael Jackson
12942 Dangerous Michael Jackson
12943 Stranger in Moscow Michael Jackson
12944 Leave Me Alone Michael Jackson
12945 In the Closet Michael Jackson
12946 Who Is It Michael Jackson
12947 Love Never Felt So Good (Original Version) Michael Jackson
12948 She’s Out of My Life Michael Jackson
12949 The Lady in My Life Michael Jackson
12950 Unbreakable Michael Jackson
12951 Ben Michael Jackson
12952 Butterflies Michael Jackson
12953 Blood on the Dance Floor Michael Jackson
12954 Baby Be Mine Michael Jackson
12955 Ghosts Michael Jackson
12956 Another Part of Me Michael Jackson
12957 H

13180 Miss Me Drake
13181 4PM in Calabasas Drake
13182 Hype Drake
13183 30 for 30 Freestyle Drake
13184 Good Ones Go (Interlude) Drake
13185 Make Me Proud Drake
13186 You & The 6 Drake
13187 Lose You Drake
13188 Buried Alive Interlude Drake
13189 Madonna Drake
13190 Over Drake
13191 Still Here Drake
13192 Now & Forever Drake
13193 Knife Talk Drake
13194 Lemon Pepper Freestyle Drake
13195 We’ll Be Fine Drake
13196 Girls Want Girls Drake
13197 Sweeterman (Remix) Drake
13198 Ice Melts Drake
13199 The Ride Drake
13200 No Long Talk Drake
13201 TSU Drake
13202 Club Paradise Drake
13203 Light Up Drake
13204 8 Out of 10 Drake
13205 That’s How You Feel Drake
13206 Elevate Drake
13207 4422 Drake
13208 Behind Barz (Link Up TV Freestyle) Drake
13209 Pain 1993 Drake
13210 Survival Drake
13211 Talk Up Drake
13212 Love All Drake
13213 Champagne Poetry Drake
13214 7am on Bridle Path Drake
13215 Can I Drake
13216 When To Say When Drake
13217 Can’t Take a Joke Drake
13218 Can’t Have Everything Drake
132

13458 Do What You Do (Remix) Drake
13459 Teach U a Lesson (Freestyle) Drake
13460 OVO Sound Radio Episode 52 Tracklist Drake
13461 One Dance (Dub) Drake
13462 Swagger Like Us Drake
13463 OVO Sound Radio Episode 25 Tracklist Drake
13464 OVO Sound Radio Episode 63 Tracklist Drake
13465 OVO Sound Radio Episode 11 Tracklist Drake
13466 OVO Sound Radio Episode 57 Tracklist Drake
13467 OVO Sound Radio Episode 24 Tracklist Drake
13468 OVO Sound Radio Episode 26 Tracklist Drake
13469 Tim Westwood Freestyle 2012 Drake
13470 OVO Sound Radio Episode 58 Tracklist Drake
13471 OVO Sound Radio Episode 46 Tracklist Drake
13472 ESPYS: Honorable Mention Drake
13473 What Up Drake
13474 OVO Sound Radio Episode 28 Tracklist Drake
13475 OVO Sound Radio Episode 16 Tracklist Drake
13476 Greatness Drake
13477 OVO Sound Radio Episode 64 Tracklist Drake
13478 OVO Sound Radio Episode 51 Tracklist Drake
13479 Madonna (Physical CD Version) Drake
13480 Throw It in the Bag (Remix) Drake
13481 Easy to Please Drake
134

13720 Leaving New York R.E.M.
13721 Bang and Blame R.E.M.
13722 Pilgrimage R.E.M.
13723 Leave R.E.M.
13724 Überlin R.E.M.
13725 Life and How to Live It R.E.M.
13726 Exhuming McCarthy R.E.M.
13727 Near Wild Heaven R.E.M.
13728 Tongue R.E.M.
13729 Texarkana R.E.M.
13730 Cuyahoga R.E.M.
13731 Gardening at Night R.E.M.
13732 Finest Worksong R.E.M.
13733 Low R.E.M.
13734 Me in Honey R.E.M.
13735 Swan Swan H R.E.M.
13736 Belong R.E.M.
13737 (I Am) Superman R.E.M.
13738 Pop Song 89 R.E.M.
13739 These Days R.E.M.
13740 Moral Kiosk R.E.M.
13741 I Believe R.E.M.
13742 7 Chinese Bros. R.E.M.
13743 Camera R.E.M.
13744 Pretty Persuasion R.E.M.
13745 Cant Get There from Here R.E.M.
13746 Catapult R.E.M.
13747 Be Mine R.E.M.
13748 Disturbance at the Heron House R.E.M.
13749 Welcome to the Occupation R.E.M.
13750 Star 69 R.E.M.
13751 9-9 R.E.M.
13752 Shaking Through R.E.M.
13753 The Flowers of Guatemala R.E.M.
13754 I Don’t Sleep, I Dream R.E.M.
13755 Lotus R.E.M.
13756 Maps and Legends R.E.M.


,Id,Title,Artist,Page Views,Pyongs Count,Annotation Count,Lyrics
0,109096,Creep,Radiohead,1791218,242.0,7,"""When you were here before\nCouldn't look you ..."
1,64344,Karma Police,Radiohead,626966,98.0,9,"""Karma police, arrest this man\nHe talks in ma..."
2,71498,No Surprises,Radiohead,597396,88.0,10,"""A heart that's full up like a landfill\nA job..."
3,78831,Paranoid Android,Radiohead,539192,105.0,9,"""Please, could you stop the noise?\nI'm trying..."
4,107387,Exit Music (For A Film),Radiohead,472639,80.0,13,"""Wake from your sleep\nThe drying of your tear..."


In [39]:
res = session.request('GET', 'https://api.genius.com/songs/2843755', headers=authorization_header).json()

In [41]:
res['response']['song']

{'annotation_count': 3,
 'api_path': '/songs/2843755',
 'apple_music_id': None,
 'apple_music_player_url': 'https://genius.com/songs/2843755/apple_music_player',
 'artist_names': 'Frank Ocean',
 'description': {'dom': {'tag': 'root',
   'children': [{'tag': 'p', 'children': ['?']}]}},
 'embed_content': "<div id='rg_embed_link_2843755' class='rg_embed_link' data-song-id='2843755'>Read <a href='https://genius.com/Frank-ocean-godspeed-screenplay-episode-1-scene-1-annotated'>“Godspeed Screenplay (Episode 1, Scene 1)” by Frank\xa0Ocean</a> on Genius</div> <script crossorigin src='//genius.com/songs/2843755/embed.js'></script>",
 'featured_video': False,
 'full_title': 'Godspeed Screenplay (Episode 1, Scene 1) by\xa0Frank\xa0Ocean',
 'header_image_thumbnail_url': 'https://images.genius.com/9491d0ed6473080fd7b62da1568da37b.300x300x1.jpg',
 'header_image_url': 'https://images.genius.com/9491d0ed6473080fd7b62da1568da37b.700x700x1.jpg',
 'id': 2843755,
 'lyrics_owner_id': 104344,
 'lyrics_placeh

In [73]:
session.request('GET', 'https://api.genius.com/search/', params={'q': "1999 prince"}, headers=authorization_header).json()

{'meta': {'status': 200},
 'response': {'hits': [{'highlights': [],
    'index': 'song',
    'type': 'song',
    'result': {'annotation_count': 18,
     'api_path': '/songs/2770',
     'artist_names': 'Prince',
     'full_title': '1999 by\xa0Prince',
     'header_image_thumbnail_url': 'https://images.genius.com/f9d763e693ae440f5cc5a95ba74a6809.300x300x1.jpg',
     'header_image_url': 'https://images.genius.com/f9d763e693ae440f5cc5a95ba74a6809.1000x1000x1.jpg',
     'id': 2770,
     'lyrics_owner_id': 7,
     'lyrics_state': 'complete',
     'path': '/Prince-1999-lyrics',
     'pyongs_count': 14,
     'song_art_image_thumbnail_url': 'https://images.genius.com/05106ba984ad1810e9043d575d8d5d95.300x292x1.jpg',
     'song_art_image_url': 'https://images.genius.com/05106ba984ad1810e9043d575d8d5d95.600x583x1.jpg',
     'stats': {'unreviewed_annotations': 3, 'hot': False, 'pageviews': 240733},
     'title': '1999',
     'title_with_featured': '1999',
     'url': 'https://genius.com/Prince-1999

In [175]:
print(getArtistId('David Bowie'))
uri = 'https://api.genius.com/artists/{}/songs'.format(getArtistId('David Bowie'))
params = {'sort': 'popularity', 'per_page': 50, 'page': 1}
response = session.request('GET', uri, params=params, headers=authorization_header)
print([song for song in response.json()['response']['songs'] if song['primary_artist']['name'] == "David Bowie"][0])

-1


KeyError: 'response'

In [166]:
print([song for song in response.json()['response']['songs'] if song['primary_artist']['name'] == "Aphex Twin"][0])

{'annotation_count': 2, 'api_path': '/songs/128396', 'artist_names': 'Aphex Twin', 'full_title': 'Avril 14th by\xa0Aphex\xa0Twin', 'header_image_thumbnail_url': 'https://images.genius.com/768f27a906ebe53e4f98398167fb84f0.300x300x1.png', 'header_image_url': 'https://images.genius.com/768f27a906ebe53e4f98398167fb84f0.1000x1000x1.png', 'id': 128396, 'lyrics_owner_id': 116340, 'lyrics_state': 'complete', 'path': '/Aphex-twin-avril-14th-lyrics', 'pyongs_count': 4, 'song_art_image_thumbnail_url': 'https://images.genius.com/768f27a906ebe53e4f98398167fb84f0.300x300x1.png', 'song_art_image_url': 'https://images.genius.com/768f27a906ebe53e4f98398167fb84f0.1000x1000x1.png', 'stats': {'unreviewed_annotations': 0, 'hot': False, 'pageviews': 15070}, 'title': 'Avril 14th', 'title_with_featured': 'Avril 14th', 'url': 'https://genius.com/Aphex-twin-avril-14th-lyrics', 'primary_artist': {'api_path': '/artists/38515', 'header_image_url': 'https://images.genius.com/b2d3bde9ef1f7f9b72f3b74f4ebb069f.1000x42

In [82]:
len(current_df.Artist.unique())

113

In [83]:
current_df.drop_duplicates(['Title', 'Artist']).to_csv('songs2.csv')

In [71]:
df.loc[:, ~df.columns.str.contains('^Unnamed')].to_csv('songs_with_lyrics.csv')

In [73]:
df = pd.read_csv('songs_with_lyrics.csv')
df.drop('Unnamed: 0', axis=1).columns

Index(['Id', 'Title', 'Artist', 'Page Views', 'Pyongs Count',
       'Annotation Count', 'Lyrics'],
      dtype='object')

In [131]:
df = pd.read_csv('songs_with_lyrics.csv')
df['Lyrics'] = df['Lyrics'].fillna('').map(repr)
df.to_csv('songs_with_lyrics.csv')

In [129]:
from ast import literal_eval
df['Lyrics'].map(literal_eval)

0        When you were here before\nCouldn't look you i...
1        Karma police, arrest this man\nHe talks in mat...
2        A heart that's full up like a landfill\nA job ...
3        Please, could you stop the noise?\nI'm trying ...
4        Wake from your sleep\nThe drying of your tears...
                               ...                        
13752                                                     
13753                                                     
13754                                                     
13755                                                     
13756                                                     
Name: Lyrics, Length: 13757, dtype: object

In [139]:
df = pd.read_csv('songs_with_lyrics.csv').drop('Unnamed: 0', axis=1)

In [144]:
list(df.loc[df['Lyrics'] == "''"].index)

[91,
 106,
 110,
 137,
 144,
 603,
 765,
 794,
 799,
 800,
 920,
 931,
 936,
 940,
 948,
 954,
 1640,
 1690,
 1854,
 1857,
 2220,
 2263,
 2267,
 2268,
 2271,
 2552,
 2561,
 2732,
 2734,
 2736,
 2740,
 2745,
 2758,
 3233,
 3248,
 3299,
 3323,
 3329,
 3554,
 3563,
 3564,
 3578,
 3835,
 3854,
 3927,
 4064,
 4070,
 4075,
 4076,
 4078,
 4082,
 4083,
 4084,
 4086,
 4088,
 4089,
 4090,
 4138,
 4140,
 4157,
 4169,
 4269,
 4274,
 4314,
 4424,
 5264,
 5407,
 5411,
 5522,
 5551,
 5587,
 5588,
 5602,
 5623,
 5624,
 5625,
 5628,
 5629,
 5632,
 5664,
 5923,
 6041,
 6064,
 6071,
 6085,
 6174,
 6209,
 6240,
 6313,
 6314,
 6316,
 6388,
 6438,
 6443,
 6451,
 6464,
 6465,
 6466,
 6568,
 6579,
 6592,
 6610,
 7652,
 7655,
 7657,
 8170,
 8247,
 8327,
 8911,
 8920,
 8950,
 8953,
 9011,
 9017,
 9520,
 9639,
 9646,
 9664,
 9680,
 9945,
 9949,
 9952,
 9963,
 9978,
 9982,
 9999,
 10012,
 10068,
 10330,
 10418,
 10422,
 10430,
 10525,
 10534,
 10831,
 11898,
 12033,
 12037,
 12349,
 12450,
 12451,
 12452,
 12453,